In [1]:
import datetime
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt
import os

In [2]:
def disch_data(battery):
    mat = loadmat(os.path.join(mat_dir, battery + '.mat'))
    print(f'[{battery}] Total data in dataset: ', len(mat[battery][0, 0]['cycle'][0]))
    c = 0
    disdataset = []
    capacity_data = []

    for i in range(len(mat[battery][0, 0]['cycle'][0])):
        row = mat[battery][0, 0]['cycle'][0, i]
        if row['type'][0] == 'discharge':
            ambient_temperature = row['ambient_temperature'][0][0]
            date_time = datetime.datetime(int(row['time'][0][0]),
                                          int(row['time'][0][1]),
                                          int(row['time'][0][2]),
                                          int(row['time'][0][3]),
                                          int(row['time'][0][4])) + datetime.timedelta(seconds=int(row['time'][0][5]))
            data = row['data']
            capacity = data[0][0]['Capacity'][0][0]
            for j in range(len(data[0][0]['Voltage_measured'][0])):
                voltage_measured = data[0][0]['Voltage_measured'][0][j]
                current_measured = data[0][0]['Current_measured'][0][j]
                temperature_measured = data[0][0]['Temperature_measured'][0][j]
                current_load = data[0][0]['Current_load'][0][j]
                voltage_load = data[0][0]['Voltage_load'][0][j]
                time = data[0][0]['Time'][0][j]
                disdataset.append([c + 1, ambient_temperature, date_time, capacity,
                                voltage_measured, current_measured,
                                temperature_measured, current_load,
                                voltage_load, time])
                capacity_data.append([c + 1, ambient_temperature, date_time, capacity])
            c += 1
    return [pd.DataFrame(data=disdataset,
                         columns=['cycle', 'ambient_temperature', 'datetime',
                                  'capacity', 'voltage_measured',
                                  'current_measured', 'temperature_measured',
                                  'current', 'voltage', 'time']),
            pd.DataFrame(data=capacity_data,
                         columns=['cycle', 'ambient_temperature', 'datetime',
                                  'capacity'])]

In [3]:
mat_dir = './battery_data/'
save_path = './datacsv/'
os.makedirs(save_path, exist_ok=True)

battery_files = [f for f in os.listdir(mat_dir) if f.endswith('.mat')]
battery_list = [f.replace('.mat', '') for f in battery_files]
excel_save_path = f'{save_path}battery_discharge_data_all.xlsx'
writer = pd.ExcelWriter(excel_save_path, engine='xlsxwriter')

In [4]:
# Create index records
index_data = []

# Batch process all batteries
for B in battery_list:
    print(f"\nProcessing Battery: {B}")
    try:
        disdf, capacity = disch_data(B)
        
        # Add survival column
        capacity['suvival'] = capacity['capacity'] >= 1.4
        
        # Write discharge data to a separate sheet in the Excel file
        disdf.to_excel(writer, sheet_name=f'{B}_discharge', index=False)
        
        # Optionally also write the capacity data table
        # capacity.to_excel(writer, sheet_name=f'{B}_capacity', index=False)
        
        # Add to index
        index_data.append({'battery_id': B, 'sheet_name': f'{B}_discharge'})
        
    except Exception as e:
        print(f"⚠️ Error processing {B}: {e}")

# Write the index sheet at the end
index_df = pd.DataFrame(index_data)
index_df.to_excel(writer, sheet_name='index', index=False)

# Save the Excel file
writer.close()
print(f"\n✅ All data has been saved to the Excel file: {excel_save_path}")



Processing Battery: B0005
[B0005] Total data in dataset:  616

Processing Battery: B0006
[B0006] Total data in dataset:  616

Processing Battery: B0007
[B0007] Total data in dataset:  616

Processing Battery: B0018
[B0018] Total data in dataset:  319

✅ All data has been saved to the Excel file: ./datacsv/battery_discharge_data_all.xlsx


In [5]:
# Corrected file path and specified engine
file_path = './datacsv/battery_discharge_data_all.xlsx'  # double-check that this is the correct path

# Load the Excel file with specified engine
xls = pd.ExcelFile(file_path, engine='openpyxl')

# Print all available sheet names
print("Available sheets:", xls.sheet_names)

# Read the first sheet (assumed to be 'B0005')
df = xls.parse(sheet_name=xls.sheet_names[0])

# Print the first 5 rows
print("\nFirst 5 rows of the B0005 sheet:")
print(df.head())

Available sheets: ['B0005_discharge', 'B0006_discharge', 'B0007_discharge', 'B0018_discharge', 'index']

First 5 rows of the B0005 sheet:
   cycle  ambient_temperature            datetime  capacity  voltage_measured  \
0      1                   24 2008-04-02 15:25:41  1.856487          4.191492   
1      1                   24 2008-04-02 15:25:41  1.856487          4.190749   
2      1                   24 2008-04-02 15:25:41  1.856487          3.974871   
3      1                   24 2008-04-02 15:25:41  1.856487          3.951717   
4      1                   24 2008-04-02 15:25:41  1.856487          3.934352   

   current_measured  temperature_measured  current  voltage    time  
0         -0.004902             24.330034  -0.0006    0.000   0.000  
1         -0.001478             24.325993  -0.0006    4.206  16.781  
2         -2.012528             24.389085  -1.9982    3.062  35.703  
3         -2.013979             24.544752  -1.9982    3.030  53.781  
4         -2.011144      